<a href="https://colab.research.google.com/github/DuplamenteH/Case-Tecnico-Geofusion/blob/main/Case_Tecnico_Estagio_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carlos Matheus

#Importações e carregamento dos dados

In [ ]:
path2data = '/content/drive/MyDrive/Estagio/DesafioEstagioMachineLearning.csv'

In [ ]:
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV,train_test_split,cross_val_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

In [ ]:
df = pd.read_csv(path2data)
df.head()

# Análise exploratória

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Vendo a porcentagem de dados faltantes 
(6/160)*100

## Tramento os dados Faltantes

In [ ]:
df_clean = df.dropna()
df_clean

In [ ]:
df = df_clean

In [ ]:
df.info()

## Verificando a coluna rendaMedia

In [ ]:
renda = df['rendaMedia']

In [ ]:
renda.max()

In [ ]:
renda.min()

In [ ]:
renda.mean()

## Algumas Perguntas que devemos fazer:

1.   Bairros com renda media maior que 60K
2.   Bairros com a maior taxa de Domicilios A1 e A2
3.   Bairros com renda media entre 5 mil e 10 mil
4.   Bairros com renda media de 5mil ate 3 mil
5.   Bairros com renda media abaixo de 3 mil
6.   Bairros com a maior taxa de Domicilios E
7.   Ver a diferença do tamanho  da população dos bairros de alta e baixa renda.
8.   Renda Media dos bairros que tem maiores taxas de Domicilios de classe E
9.   Bairro de maior população




## 1. Bairros com renda media maior que 60K e renda acima de 10mil

In [ ]:
bairrosRenda_maior60k = df.loc[df['rendaMedia']<=1000]
bairrosRenda_maior60k.shape[0]

In [ ]:
bairrosRenda_maior60k

 **Assim podemos classificar esta linha como um "outlier"**

In [ ]:
bairrosRenda_maior10k = df.loc[df['rendaMedia']>=10000]
bairrosRenda_maior10k.shape[0]

In [ ]:
bairrosRenda_maior10k.reset_index(inplace=True)

In [ ]:
nome_bairros_renda_maior_10K = bairrosRenda_maior10k['nome']
nome_bairros_renda_maior_10K

## 2.Bairros com a maior taxa de Domicilios A1 e A2

In [ ]:
maior_taxa_DomiciliosA = df.sort_values(by = ['domiciliosA1','domiciliosA2'],ascending=False)
maior_taxa_DomiciliosA.reset_index(inplace=True)
maior_taxa_DomiciliosA.drop(columns=['index'],inplace=True)
maior_taxa_DomiciliosA['nome'].head(12)

In [ ]:
bairro_renda_Alta = maior_taxa_DomiciliosA.head(10)

In [ ]:
renda_media_Classe_Alta = bairro_renda_Alta.rendaMedia.mean()
round(renda_media_Classe_Alta)

### **O bairro com maior taxa de domicilios de classe A é a Barra da Tijuca**

## 3. Bairros com renda media entre 5 mil e 10 mil

In [ ]:
teste = ((df['rendaMedia']<=10000) & (df['rendaMedia']>=5000))
renda_entre5k_e_10k = df[teste]

In [ ]:
renda_entre5k_e_10k.shape[0]

In [ ]:
renda_entre5k_e_10k.reset_index(inplace=True)

In [ ]:
bairros_renda_entre5k_e_10k = renda_entre5k_e_10k['nome']
bairros_renda_entre5k_e_10k

## 4. Bairros com renda media de 5mil ate 3 mil

In [ ]:
selecao = ((df['rendaMedia']>=3000) & (df['rendaMedia']<=5000))
renda_entre3k_e_5k = df[selecao]

In [ ]:
renda_entre3k_e_5k.reset_index(inplace=True)

In [ ]:
renda_entre3k_e_5k.shape[0]

In [ ]:
bairros_renda_entre3k_e_5k = renda_entre3k_e_5k['nome']
bairros_renda_entre3k_e_5k

## 5.Bairros com renda media abaixo de 3 mil

In [ ]:
bairrosRenda_menor3k = df.loc[df['rendaMedia']<3000]
bairrosRenda_menor3k.shape[0]

In [ ]:
bairrosRenda_menor3k.reset_index(inplace=True)

In [ ]:
baixa_renda=bairrosRenda_menor3k.head(10)

In [ ]:
bairros_baixa_renda = bairrosRenda_menor3k['nome']

In [ ]:
bairros_baixa_renda.head(10)

In [ ]:
bairros_baixa_renda.shape[0]

## 6 e 8 .Bairros com a maior taxa de Domicilios E . / Renda Media dos bairros que tem maiores taxas de Domicilios de classe E

In [ ]:
df_copy = df.copy()

In [ ]:
df_domicilioD_E = df_copy.sort_values(by=['domiciliosE','domiciliosD'],ascending=False)
df_domicilioD_E.reset_index(inplace=True)

In [ ]:
df_domicilioD_E.drop(columns=['index'],inplace=True)

In [ ]:
df_domicilioD_E['nome'].head()

In [ ]:
top5_renda_baixa = df_domicilioD_E.head(10)

In [ ]:
renda_media_Classe_E = top5_renda_baixa.rendaMedia.mean()
round(renda_media_Classe_E)

##7. Ver a diferença de população dos bairros de alta e baixa renda

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
data = [go.Bar(x=df_domicilioD_E.população.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.população.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população nos bairros com maiores indices de domicilios de baixa renda e de alta renda',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig = go.Figure(data=data, layout=configuracoes_layout)
fig.show()

### Outro dado interessante seria a faixa etaria da população

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popAte9.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popAte9.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de ate 9 anos,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig2 = go.Figure(data=data, layout=configuracoes_layout)
fig2.show()

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popDe10a14.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popDe10a14.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de 10 a 14 anos,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig2 = go.Figure(data=data, layout=configuracoes_layout)
fig2.show()

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popDe15a19.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popDe15a19.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de 15 a 19 anos,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig3 = go.Figure(data=data, layout=configuracoes_layout)
fig3.show()

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popDe20a24.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popDe20a24.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de 20 a 24 anos,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig4 = go.Figure(data=data, layout=configuracoes_layout)
fig4.show()

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popDe25a34.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popDe25a34.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de 25 a 34 anos,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig5 = go.Figure(data=data, layout=configuracoes_layout)
fig5.show()

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popDe35a49.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popDe35a49.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de 35 a 49 anos,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig6 = go.Figure(data=data, layout=configuracoes_layout)
fig6.show()

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popDe35a49.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popDe35a49.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de 35 a 49 anos,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig7 = go.Figure(data=data, layout=configuracoes_layout)
fig7.show()

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popDe50a59.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popDe50a59.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de 50 a 59 anos,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig = go.Figure(data=data, layout=configuracoes_layout)
fig.show()

In [ ]:
data = [go.Bar(x=df_domicilioD_E.popMaisDe60.head(10),
               y=df_domicilioD_E.nome.head(10),
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Baixa renda',
               opacity= 0.7,
               orientation = 'h'
              ),
        go.Bar(x=maior_taxa_DomiciliosA.popMaisDe60.head(10),
               y=maior_taxa_DomiciliosA.nome.head(10),
               marker = {'color': 'lightgreen',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               name='Alta renda',
               opacity= 0.7,
               orientation = 'h'
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Diferença da qtd de população na faixa etária de 60 anos ou mais,  nos bairros com maiores indices de domicilios de baixa renda e de alta renda ',
                                 yaxis={'title':'População'},
                                 xaxis={'title':'bairros'})

# Objeto figura

fig = go.Figure(data=data, layout=configuracoes_layout)
fig.show()

## 9. Bairros de maior população e o Bairros com o maior faturamento

In [ ]:
populosos=df[df.população>df.população.mean()]
# Aqui faço uma query pegar os bairros que tem população acima media , para assim reduzir o tamanho do nosso dataset, de  154 para 46
populosos.shape

In [ ]:
populosos = populosos.sort_values(ascending=False,by='população')
#aqui faço uma ordenação da base decrescente a partir da população, e exibo os 5 bairros mais populosos do RJ
populosos.head()

In [ ]:
faturamente = df[df.faturamento>df.faturamento.mean()]

In [ ]:
#faço o mesmo passo a passo  que fiz para a populaçao para o faturamento.
faturamento = faturamente.sort_values(by = 'faturamento', ascending=False)
faturamento.head()

**Podemos perceber que o os 5 bairros de maior faturamento nenhum apresenta mais de 7 mil domicilios de classe baixa,já dos bairros mais populosos o que menor apresenta uma taxa domicilios de classe baixa é a Tijica com pouco mais de 4 mil**

## Análises estatística

In [ ]:
df.sort_values(by='rendaMedia',ascending=False).head()

In [ ]:
corr = df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")

In [ ]:
mask = np.triu(np.ones_like(corr, dtype=bool))

In [ ]:
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### Boxplot dos domicilios

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=df.domiciliosA1,name="Domicilios A1"))
fig.add_trace(go.Box(y=df.domiciliosA2,name="Domicilios A2"))
fig.add_trace(go.Box(y=df.domiciliosB1,name="Domicilios B1"))
fig.add_trace(go.Box(y=df.domiciliosB2,name="Domicilios B2"))
fig.add_trace(go.Box(y=df.domiciliosC1,name="Domicilios C1"))
fig.add_trace(go.Box(y=df.domiciliosC2,name="Domicilios C2"))
fig.add_trace(go.Box(y=df.domiciliosD,name="Domicilios D"))
fig.add_trace(go.Box(y=df.domiciliosE,name="Domicilios E"))

fig.show()

### Boxplot da renda media

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=df.rendaMedia,name="Renda media"))

fig.show()

In [ ]:
df_backup = df.copy()

In [ ]:
df.drop(index=[1,11,40],inplace=True)

In [ ]:
df.rendaMedia.min()

### Boxplot Faturamento

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=df.faturamento,name="Renda media"))

fig.show()

# Modelo

In [ ]:
df.columns

In [ ]:
feature = ['domiciliosA1','domiciliosA2',
       'domiciliosB1', 'domiciliosB2', 'domiciliosC1', 'domiciliosC2',
       'domiciliosD', 'domiciliosE', 'rendaMedia']

In [ ]:
X = df[feature]
X

In [ ]:
y = df['faturamento']
y

In [ ]:
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

### Fazendo nosso baseline

In [ ]:
baseline = DummyRegressor()

In [ ]:
 metricas = ['explained_variance', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error', 'r2']

In [ ]:
baseline.get_params().keys()

In [ ]:
params = {
    'strategy':['mean','median','quartile','constant'],
    'quantile':[.75],
    'constant':[300000]
}

In [ ]:
grid_baseline = GridSearchCV(baseline,param_grid=params, scoring=metricas, verbose=100, refit='neg_mean_squared_error',return_train_score=False)

In [ ]:
grid_baseline.fit(X_train,y_train)

In [ ]:
grid_baseline.best_estimator_

In [ ]:
np.sqrt(grid_baseline.best_score_ * -1)

In [ ]:
pd.set_option('max_columns',200)
grid_df_baseline=pd.DataFrame(grid_baseline.cv_results_)

In [ ]:
grid_df_baseline

### Linear Regression

In [ ]:
reg_linear = LinearRegression()

In [ ]:
reg_linear.get_params().keys()

In [ ]:
params_reg = {
    'fit_intercept':[True,False],
    'normalize':[True,False]
}

In [ ]:
grid_reg_linear = GridSearchCV(reg_linear, param_grid=params_reg,cv=10 ,scoring=metricas, verbose=100, refit='neg_mean_squared_error', return_train_score=False)

In [ ]:
grid_reg_linear.fit(X_train,y_train)

In [ ]:
grid_reg_linear.best_estimator_

In [ ]:
linear_best_score = np.sqrt(grid_reg_linear.best_score_*-1)

In [ ]:
linear_best_score

In [ ]:
pd.DataFrame(grid_reg_linear.cv_results_)

### Decision Tree Regressor

In [ ]:
tree_reg = DecisionTreeRegressor()

In [ ]:
tree_reg.get_params().keys()

In [ ]:
params_tree ={
            "splitter":["best","random"],
            "max_depth" : [1,3,5,7,9,11,12],
            "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
            "max_features":["auto","log2","sqrt"],
            "max_leaf_nodes":[10,20,30,40,50,60,70,80,90] 
            }

In [ ]:
grid_tree_reg = GridSearchCV(tree_reg,param_grid=params_tree,cv=5,scoring=metricas, verbose=50, refit='neg_mean_squared_error', return_train_score=False)

In [ ]:
grid_tree_reg.fit(X_train,y_train)

In [ ]:
grid_tree_reg.best_estimator_

In [ ]:
np.sqrt(grid_tree_reg.best_score_*-1)

### Random Florest

In [ ]:
params_randomF = {
    'bootstrap': [True,False],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


rF = RandomForestRegressor()

In [ ]:
grid_random_florest = GridSearchCV(rF,param_grid=params_randomF,scoring=metricas,cv = 3, n_jobs = -1, verbose = 50,refit='neg_mean_squared_error', return_train_score=False)

In [ ]:
grid_random_florest.fit(X_train,y_train)

In [ ]:
grid_random_florest.best_estimator_

In [ ]:
grid_score_random_florest = np.sqrt(grid_random_florest.best_score_*-1)
grid_score_random_florest 

### Gradiente boosting regressor

In [ ]:
gradiente_params = {
    'n_estimators':[500,1000,2000],
    'learning_rate':[.001,0.01,.1],
    'max_depth':[1,2,4,6,8,10],
    'subsample':[.5,.75,1],
    'random_state':[42]
}

gbr = GradientBoostingRegressor()

In [ ]:
grid_gbr = GridSearchCV(gbr,param_grid=gradiente_params,scoring=metricas,n_jobs=5,cv=10,refit='neg_mean_squared_error',verbose = 50, return_train_score=False)

In [ ]:
grid_gbr.fit(X_train,y_train)

In [ ]:
grid_gbr.best_estimator_

In [ ]:
np.sqrt(grid_gbr.best_score_*-1)

### Olhando os scores

In [ ]:
score_reg_linear = np.sqrt(grid_reg_linear.best_score_*-1)
score_decision_tree_regr = np.sqrt(grid_tree_reg.best_score_*-1)
score_random_florest = grid_score_random_florest 
score_gbr = np.sqrt(grid_gbr.best_score_*-1)

In [ ]:
display_score(grid_reg_linear.score(X_test,y_test))

In [ ]:
scores_train = {
    'reg_linear_train': [score_reg_linear],
    'decision_tree_regr_train': [score_decision_tree_regr],
    'random_florest_train': [score_random_florest],
    'score_gbr_train':[score_gbr]
}

In [ ]:
df_score_train = pd.DataFrame(data=scores_train,dtype=float,index=pd.RangeIndex(start=0,step=1,stop=1))
df_score_train

In [ ]:
scores_test = {
    'reg_linear_score_test':[round(np.sqrt(grid_reg_linear.score(X_test,y_test)*-1))],
    'decision_tree_regr_score_test':[round(np.sqrt(grid_tree_reg.score(X_test,y_test)*-1))],
    'random_florest_score_test':[round(np.sqrt(grid_random_florest.score(X_test,y_test)*-1))],
    'gbr_score_test':[round(np.sqrt(grid_gbr.score(X_test,y_test)*-1))]
}

In [ ]:
df_score_test = pd.DataFrame(data=scores_test,dtype=float,index=pd.RangeIndex(start=0,step=1,stop=1))
df_score_test

### Bom podemos ver que o dentre os 4 algoritmos escolhidos o que saiu melhor é o gradiente boosting regressor .

In [ ]:
grid_gbr.best_params_

In [ ]:
gbr = GradientBoostingRegressor(learning_rate=0.1,max_depth=1,n_estimators=2000,random_state=42,subsample=1)


In [ ]:
gbr.fit(X_train,y_train)

In [ ]:
gbr.score(X_test,y_test)*100

In [ ]:
X.columns

In [ ]:
teste = X_test.sample()

In [ ]:
teste

In [ ]:
gbr.predict(teste)

In [ ]:
y_test

### Expotando o modelo

In [ ]:
import pickle

In [ ]:
with open('predict_gbr.pkl', 'wb') as file:
    pickle.dump(gbr, file)